In [0]:
# Databricks notebook source code
from pyspark.sql.functions import col, struct, from_json
from pyspark.sql.types import StructField, StructType, StringType

In [0]:
df = spark \
     .readStream \
     .format("kafka") \
     .option("kafka.bootstrap.servers", "SERVERNAME-HERE") \
     .option("subscribe", "topic_name") \
     .option("maxOffsetsPerTrigger", "10")\
     .option("kafka.security.protocol","SASL_SSL") \
     .option("kafka.sasl.mechanism", "PLAIN") \
     .option("kafka.sasl.jaas.config", """kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="USERNAME-HERE" password="PASSWORD-HERE";""") \
    .load()

In [0]:
source_file_schema = StructType(
  [
    StructField("customer_id", StringType(), True),
    StructField("month", StringType(), True),
    StructField("category", StringType(), True),
    StructField("payment_type", StringType(), True),
    StructField("spend", StringType(), True),
    StructField("transaction_id", StringType(), True)
  ]
)

In [0]:
df_data = df.withColumn(
    "value", from_json(col("value").cast("string"), source_file_schema)
).select(
    col("value.customer_id"),
    col("value.month"),
    col("value.category"),
    col("value.payment_type"),
    col("value.spend"),
    col("value.transaction_id")
)


In [0]:
spark.conf.set(
    "fs.azure.account.key.<storage-account-name>.dfs.core.windows.net",
    "<storage-acount-key>"
)

In [0]:
output_path = 'abfss://<container-name>@<stroarge-account-name>.dfs.core.windows.net/sink'
checkpoint_location = 'abfss://<container-name>@<stroarge-account-name>.dfs.core.windows.net/checkpoint'

streaming_query = (
    df_data.writeStream
    .format("csv")
    .option("path", output_path)
    .option("header", True)
    .option("checkpointLocation", checkpoint_location)
    .start()
)

display(df_data)
